In [1]:
month = 'Febrero'
filename = '2022_09_Nadro.xlsx'

# Libraries

In [2]:
import pandas as pd
from datetime import datetime

# Parameters

In [6]:
year = datetime.today().year
#week = datetime.today().isocalendar()[1]
week = 9

path_export = r'\\NASPRO.infovisiontv.com\Respaldo_Operacion_Cargas\Nadro\{0}\Layout Diario'
path_export = path_export.format(year)

path_backup = r'\\NASPRO.infovisiontv.com\Respaldo_Operacion_Cargas\Nadro\{0}'
path_backup = path_backup.format(year)

path_load = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\MEX\Carga\Semanales\Nadro\Data\{0}\2. {2}\S {1}'
path_load = path_load.format(year, str(week).zfill(2), month)

 = r'\\NASPRO.infovisiontv.com\Respaldo_Operacion_Cargas\Nadro\Catalogos.xlsx'
path_inv = r'\\NASPRO.infovisiontv.com\Respaldo_Operacion_Cargas\Nadro\Catalogos.xlsx'

# Load Data

In [4]:
df = pd.read_excel(path_load + '\\' + filename)

In [9]:
dsucs = pd.read_excel(path_sucs, sheet_name='Sucursales', names=['Suc', 'Suc_Num', 'Nombre', 'SUCURSAL'])

In [12]:
dinv = pd.read_excel(r'\\NASPRO.infovisiontv.com\Respaldo_Operacion_Cargas\Nadro\Catalogos.xlsx', sheet_name='Inventarios')

# Data Cleaning

## Remove data

In [5]:
del df['TRANSF']
del df['PRON']

df = df[:-1]

In [15]:
df.columns

Index(['EAN', 'SAP', 'MATERIAL', 'MARCA', 'EMPQ', 'PFCIA', 'PPUB', 'CS',
       'CTRO', 'SUCURSAL', 'DEMANDA', 'EXIS', 'TRANS', 'LUNES 21', 'MARTES 22',
       'MIERCOLES 23', 'JUEVES 24', 'VIERNES 25', 'DOMINGO 27', 'TOT PZAS',
       'TOT VTA P FCIA'],
      dtype='object')

rename_cols = {
    'MATERIAL': 'DESCRIPCION',
    'CTRO': 'SUCURSAL',
    'SUCURSAL': 'DSC_SUCURSAL',
    'SAP': 'CODIGO',
    'EMPQ': 'EMPAQUE',
    'EXIS': 'EXISTENCIA',
    'TRANS': 'TRANSITO'
    }

df.rename(rename_cols, axis=1, inplace=True)

df['DES_CS'] = 'Disponible'

sort_cols = [
    'EAN',
    'DESCRIPCION',
    'MARCA',
    'SUCURSAL',
    'DSC_SUCURSAL',
    'CODIGO',
    'CS',
    'DES_CS',
    'EMPAQUE',
    'PFCIA',
    'PPUB',
    'EXISTENCIA',
    'TRANSITO',
    'DEMANDA'
    ]